In [1]:
!pip install rank_bm25 pypdf pdfplumber sentence-transformers faiss-cpu pandas transformers tabulate scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [2]:
import re
import numpy as np
import pdfplumber
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

2025-07-28 03:40:18.229102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753674018.390376      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753674018.439602      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# -------- TEXT CLEANING & EXTRACTION ----------
def extract_clean_text(pdf_path):
    """
    Extracts and cleans text from a PDF financial report by:
    1. Removing boilerplate sections (disclaimers, contacts)
    2. Normalizing whitespace
    3. Preserving the core financial content
    
    Args:
        pdf_path (str): Path to the PDF file
        
    Returns:
        str: Cleaned text content
    """
    text = ""
    # Extract text page by page
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    
    # Remove common financial report boilerplate
    text = re.sub(r'Forward-Looking Statements.*', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'Contacts Investors:.*', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'About Meta.*', '', text, flags=re.DOTALL|re.IGNORECASE)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [4]:
# --------- SMART CHUNKING -----------
def smart_chunk_text(text, chunk_size=80, overlap=20):
    """
    Splits text into coherent chunks by:
    - Breaking at sentence boundaries
    - Maintaining context with overlap between chunks
    - Enforcing maximum token limits per chunk
    
    Args:
        text (str): Input text to chunk
        chunk_size (int): Max words per chunk
        overlap (int): Words to overlap between chunks
        
    Returns:
        list: List of text chunks
    """
    # Split into sentences using punctuation
    sents = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current = []
    tokens = 0
    
    for s in sents:
        s_tok = len(s.split())
        
        # Create new chunk if adding this sentence would exceed limit
        if tokens + s_tok > chunk_size and current:
            chunks.append(' '.join(current))
            # Keep overlap between chunks
            current = current[-overlap:] if overlap else []
            tokens = sum(len(x.split()) for x in current)
            
        current.append(s)
        tokens += s_tok
    
    # Add final chunk
    if current:
        chunks.append(' '.join(current))
        
    return chunks

# --------- EMBEDDING/INDEX ----------
def get_embeddings(chunks, model_name="all-MiniLM-L6-v2"):
    """
    Generates vector embeddings for text chunks using SentenceTransformer
    
    Args:
        chunks (list): Text chunks to embed
        model_name (str): Embedding model to use
        
    Returns:
        tuple: (embeddings numpy array, embedding model)
    """
    embedder = SentenceTransformer(model_name)
    embs = embedder.encode(chunks, show_progress_bar=True).astype(np.float32)
    return embs, embedder

def build_faiss_index(embeddings):
    """
    Creates a FAISS index for efficient similarity search
    
    Args:
        embeddings (numpy.array): Array of vector embeddings
        
    Returns:
        faiss.Index: Searchable index
    """
    # Create a flat L2 (Euclidean) distance index
    idx = faiss.IndexFlatL2(embeddings.shape[1])
    idx.add(embeddings)
    return idx

def retrieve(query, embedder, faiss_index, chunks, top_k=3):
    """
    Retrieves most relevant text chunks for a query using semantic search
    
    Args:
        query (str): User question
        embedder: SentenceTransformer model
        faiss_index: FAISS index
        chunks (list): Original text chunks
        top_k (int): Number of chunks to return
        
    Returns:
        list: Top matching chunks
    """
    # Encode query and search index
    q_emb = embedder.encode([query]).astype(np.float32)
    _, idxs = faiss_index.search(q_emb, top_k)
    
    # Return actual text chunks
    return [chunks[i] for i in idxs[0]]

In [5]:
# --------- LLM GENERATION ----------
def gen_answer(prompt):
    """
    Generates answers using TinyLlama 1.1B model with:
    - Meta financial analyst persona
    - Concise response style
    - Context-aware generation
    
    Args:
        prompt (str): Full prompt including context and question
        
    Returns:
        str: Generated answer
    """
    model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    
    # Initialize model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
    
    # Create text generation pipeline
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    
    # Generate and clean output
    output = pipe(prompt, max_new_tokens=200, do_sample=False)[0]['generated_text']
    return output[len(prompt):].strip()

In [6]:
# --------- MAIN PIPELINE -----------
def basic_rag_pipeline(pdf_path, queries):
    """
    Complete RAG pipeline execution:
    1. PDF text extraction
    2. Chunking
    3. Embedding and indexing
    4. Question answering loop
    
    Args:
        pdf_path (str): Path to financial report PDF
        queries (list): List of questions to answer
    """
    print("Extracting and cleaning PDF...")
    text = extract_clean_text(pdf_path)
    
    print("Creating text chunks...")
    chunks = smart_chunk_text(text)
    print(f"Total chunks: {len(chunks)}")
    
    print("Generating embeddings...")
    embeddings, embedder = get_embeddings(chunks)
    
    print("Building search index...")
    idx = build_faiss_index(embeddings)
    
    # Process each question
    for q in queries:
        print(f"\nQ: {q}")
        
        # Retrieve relevant context
        context = '\n'.join(retrieve(q, embedder, idx, chunks, top_k=3))
        
        # Create LLM prompt
        prompt = f"""Based on the following financial report context, answer concisely as a Meta financial analyst.
        Context:
        {context}
        Question: {q}
        Answer:"""
        
        # Generate and display answer
        answer = gen_answer(prompt)
        print(f"Answer:\n{answer}\n")

In [7]:
if __name__ == "__main__":
    # Sample financial report and questions
    PDF = "/kaggle/input/financial-reporta/Metas Q1 2024 Financial Report.pdf"
    
    questions = [
        "When did Meta report its first quarter 2024 results?",
        "What was Meta's total revenue for Q1 2024?",
        "How much did Meta's revenue increase compared to Q1 2023?",
        "What were the total costs and expenses for Meta in Q1 2024?",
        "How much income from operations did Meta report in Q1 2024?",
        "What was Meta's operating margin in Q1 2024?",
        "How much provision for income taxes did Meta have in Q1 2024?",
        "What was Meta's effective tax rate in Q1 2024?",
        "How much net income did Meta earn in Q1 2024?",
        "What was Meta's diluted earnings per share (EPS) for Q1 2024?",
        "What was the amount paid as dividends in Q1 2024?",
        "What revenue range does Meta expect for Q2 2024?",
        "What is the expected effect of foreign currency on Meta's revenue growth?",
    ]
    
    # Run the full pipeline
    basic_rag_pipeline(PDF, questions)

Extracting and cleaning PDF...
Creating text chunks...
Total chunks: 29
Generating embeddings...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer:
Meta expects revenue in the range of $1.25 billion to $1.3 billion for Q2 2024.


Q: What is the expected effect of foreign currency on Meta's revenue growth?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Answer:
The expected effect of foreign currency on Meta's revenue growth is a 1% headwind to year-over-year total revenue growth, based on current exchange rates.

